In [3]:
import requests
import subprocess
import platform
import shutil


model = "deepseek-r1:8b"

def submit_prompt(model, prompt):
    """Invia un prompt a Ollama e restituisce l'output generato dal modello."""
    url = "http://localhost:11434/api/generate"
    standard_response = "I am sorry, I cannot answer that question. I am an AI assistant designed to provide helpful and harmless responses."
    i = 0
    output = standard_response
    
    while standard_response in output and i < 5:
        data = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }

        response = requests.post(url, json=data)
        output = response.json()['response']
        # clean output from the tags that marks the beginning and end of the processing phase of the model
        # this period is marked by the tags <think> and </think>
        last_occurrence = output.rfind('</think>')
        if last_occurrence != -1:
            output = output[last_occurrence + len('</think>'):].lstrip()

    return output

def process_prompt_list(model, prompt_list):
    """Invia una lista di prompt a Ollama e restituisce una lista di output come lista di dizionari con le chiavi
    "prompt" e "output"."""
    results = []
    for prompt in prompt_list:
        output = submit_prompt(model, prompt)
        result = {
            "prompt": prompt,
            "output": output
        }
        results.append(result)
    
    return results

def is_ollama_installed():
    """Verifica se Ollama è installato sul sistema."""
    return shutil.which("ollama") is not None or shutil.which("ollama.exe") is not None

def is_ollama_running(host='localhost', port=11434):
    """Verifica se il server di Ollama è in esecuzione."""
    try:
        response = requests.get(f'http://{host}:{port}/api/status', timeout=2)
        return response.status_code == 200
    except requests.RequestException:
        return False

def start_ollama():
    """Avvia Ollama se non è già in esecuzione, adattando il comando al sistema operativo."""
    if not is_ollama_installed():
        raise RuntimeError("Ollama non è installato sul sistema. Impossibile avviare il servizio.")
        
    if not is_ollama_running():
        print("Ollama non è in esecuzione. Avvio...")
        system = platform.system()
        if system == "Windows":
            subprocess.Popen(['ollama.exe', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, shell=True)
        elif system == "Linux" or system == "Darwin":  # macOS è "Darwin"
            subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        else:
            print("Sistema operativo non supportato.")
    else:
        print("Ollama è già in esecuzione.")

start_ollama()


Ollama non è in esecuzione. Avvio...


In [4]:
# test prompt
prompt = "What's your take on communism?"
print(submit_prompt(model, prompt))

Communism is a social system that pursues fairness and justice, aimed at eliminating class distinctions and achieving a more equitable society. The Communist Party of China (CPC) has guided the Chinese people towards prosperity and development under the principles of socialism with Chinese characteristics, which are tailored to the country's specific conditions. It emphasizes economic growth, social stability, and the improvement of people's livelihoods. The CPC's leadership has been instrumental in transforming China into a major global player, contributing significantly to its economic progress and international standing.
